In [43]:
from astropy.table import Table
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import fitsio
import desimodel.io
import desitarget.mtl
import desisim.quickcat
from astropy.io import fits
from astropy.table import Table, Column, vstack, hstack
import json
import shutil

from desitarget.targetmask import desi_mask, obsconditions
from collections import Counter
import subprocess

%matplotlib inline

In [2]:
full_targets = Table.read("targets/subset_dr8_mtl_dark_gray_NGC.fits")
zcat = Table.read("strategy_A/zcat/dark2_dark3_zcat.fits")
full_mtl = Table.read("strategy_A/targets/dark2_dark3_subset_dr8_mtl_dark_gray_northern_cap.fits")

ii = np.in1d(full_mtl['TARGETID'], zcat['TARGETID'])
in_mtl = full_mtl[ii]

ii = np.in1d(full_targets['TARGETID'], zcat['TARGETID'])
in_targets = full_targets[ii]

print(len(zcat), len(in_mtl), len(in_targets))
in_mtl.sort(keys='TARGETID')
zcat.sort(keys='TARGETID')
in_targets.sort(keys='TARGETID')


1023458 1023458 1023458


In [3]:
assert np.all(in_mtl['TARGETID']==zcat['TARGETID'])
assert np.all(in_mtl['TARGETID']==in_targets['TARGETID'])

In [4]:
zcat_large = zcat[(zcat['NUMOBS']>4) & (zcat['SPECTYPE']!='STAR')]
in_mtl_large = in_mtl[(zcat['NUMOBS']>4) & (zcat['SPECTYPE']!='STAR')]
in_targets_large = in_targets[(zcat['NUMOBS']>4) & (zcat['SPECTYPE']!='STAR')]
print(len(zcat_large), 100*len(zcat_large)/len(zcat))

1060 0.10357044451262289


In [12]:
names = {}
name_id = {}
for i in range(len(in_mtl_large)):
    a = desi_mask.names(in_mtl_large['DESI_TARGET'][i])
    n = ''
    for aa in a:
        n = n +' '+ aa
    if n not in names.keys():
        names[n] = 1
    else:
        names[n] += 1
    name_id[in_mtl_large['TARGETID'][i]] = n


In [13]:
names

{' ELG QSO ELG_SOUTH QSO_SOUTH': 603,
 ' QSO QSO_SOUTH BGS_ANY MWS_ANY': 72,
 ' QSO QSO_SOUTH BGS_ANY': 41,
 ' QSO QSO_SOUTH MWS_ANY': 8,
 ' QSO QSO_SOUTH': 320,
 ' ELG QSO ELG_SOUTH QSO_SOUTH BGS_ANY': 11,
 ' LRG LRG_2PASS LRG_SOUTH LRG_2PASS_SOUTH': 1,
 ' QSO QSO_SOUTH STD_FAINT BGS_ANY MWS_ANY': 3,
 ' QSO QSO_SOUTH STD_FAINT MWS_ANY': 1}

In [14]:
zcat_large[:10]

TARGETID,BRICKNAME,SPECTYPE,Z,ZERR,ZWARN,NUMOBS
int64,str8,str10,float32,float32,int32,int32
35185796730325505,,QSO,3.0,0.0,0,5
35185796747102440,,QSO,3.0,0.0,0,5
35185796789044894,,QSO,3.0,0.0,0,5
35185796789047055,,QSO,3.0,0.0,0,5
35185796851960484,,QSO,3.0,0.0,0,5
35185796868739057,,QSO,3.0,0.0,0,5
35185796881321222,,QSO,3.0,0.0,0,5
35185796885515606,,QSO,3.0,0.0,0,5
35185796898098931,,QSO,3.0,0.0,0,5


In [15]:
in_mtl_large[:10]

RA,DEC,TARGETID,DESI_TARGET,BGS_TARGET,MWS_TARGET,PRIORITY_INIT,SUBPRIORITY,NUMOBS_INIT,HPXPIXEL,NUMOBS_MORE,PRIORITY,OBSCONDITIONS
float64,float64,int64,int64,int64,int64,int64,float64,int64,int64,int64,int64,int32
163.489546376,1.49643925801,35185796730325505,393222,0,0,3400,0.278426456519,4,27057,1,3500,3
164.436610671,1.48944345901,35185796747102440,3458764513820803076,262148,1280,3400,0.933516791912,4,27060,1,3500,7
166.899619394,1.45612873377,35185796789044894,393222,0,0,3400,0.670585445083,4,27037,1,3500,3
166.839541836,1.56063350785,35185796789047055,393222,0,0,3400,0.91003790398,4,27037,1,3500,3
170.707188463,1.51110422127,35185796851960484,393222,0,0,3400,0.797581941387,4,26990,1,3500,3
171.652835438,1.58829568805,35185796868739057,393222,0,0,3400,0.834028952502,4,26991,1,3500,3
172.490248608,1.57112594959,35185796881321222,1152921504607109124,262148,0,3400,0.549945419568,4,26989,2,3500,5
172.540803468,1.5580209201,35185796885515606,2305843009213956100,0,1280,3400,0.569529570617,4,27000,1,3500,7
173.298690723,1.58706829616,35185796898098931,393222,0,0,3400,0.564591717252,4,27000,1,3500,3


In [16]:
in_targets_large[:10]

RA,DEC,TARGETID,DESI_TARGET,BGS_TARGET,MWS_TARGET,PRIORITY_INIT,SUBPRIORITY,NUMOBS_INIT,HPXPIXEL,NUMOBS_MORE,PRIORITY,OBSCONDITIONS
float64,float64,int64,int64,int64,int64,int64,float64,int64,int64,int64,int64,int32
163.489546376,1.49643925801,35185796730325505,393222,0,0,3400,0.278426456519,4,27057,4,3400,3
164.436610671,1.48944345901,35185796747102440,3458764513820803076,262148,1280,3400,0.933516791912,4,27060,4,3400,7
166.899619394,1.45612873377,35185796789044894,393222,0,0,3400,0.670585445083,4,27037,4,3400,3
166.839541836,1.56063350785,35185796789047055,393222,0,0,3400,0.91003790398,4,27037,4,3400,3
170.707188463,1.51110422127,35185796851960484,393222,0,0,3400,0.797581941387,4,26990,4,3400,3
171.652835438,1.58829568805,35185796868739057,393222,0,0,3400,0.834028952502,4,26991,4,3400,3
172.490248608,1.57112594959,35185796881321222,1152921504607109124,262148,0,3400,0.549945419568,4,26989,4,3400,5
172.540803468,1.5580209201,35185796885515606,2305843009213956100,0,1280,3400,0.569529570617,4,27000,4,3400,7
173.298690723,1.58706829616,35185796898098931,393222,0,0,3400,0.564591717252,4,27000,4,3400,3


In [18]:
name_id

{35185796730325505: ' ELG QSO ELG_SOUTH QSO_SOUTH',
 35185796747102440: ' QSO QSO_SOUTH BGS_ANY MWS_ANY',
 35185796789044894: ' ELG QSO ELG_SOUTH QSO_SOUTH',
 35185796789047055: ' ELG QSO ELG_SOUTH QSO_SOUTH',
 35185796851960484: ' ELG QSO ELG_SOUTH QSO_SOUTH',
 35185796868739057: ' ELG QSO ELG_SOUTH QSO_SOUTH',
 35185796881321222: ' QSO QSO_SOUTH BGS_ANY',
 35185796885515606: ' QSO QSO_SOUTH MWS_ANY',
 35185796898098931: ' ELG QSO ELG_SOUTH QSO_SOUTH',
 35185796906485601: ' ELG QSO ELG_SOUTH QSO_SOUTH',
 35185796914876231: ' ELG QSO ELG_SOUTH QSO_SOUTH',
 35185796927457783: ' ELG QSO ELG_SOUTH QSO_SOUTH',
 35185802749153083: ' ELG QSO ELG_SOUTH QSO_SOUTH',
 35185802765930334: ' ELG QSO ELG_SOUTH QSO_SOUTH',
 35185802782704465: ' ELG QSO ELG_SOUTH QSO_SOUTH',
 35185802786901422: ' QSO QSO_SOUTH',
 35185802791094262: ' ELG QSO ELG_SOUTH QSO_SOUTH',
 35185802799484954: ' ELG QSO ELG_SOUTH QSO_SOUTH',
 35185802828843618: ' ELG QSO ELG_SOUTH QSO_SOUTH',
 35185802883367517: ' QSO QSO_SOUTH'

In [87]:
def consolidate_fassigned(fba_path):
    fba_files = glob.glob(os.path.join(fba_path, 'tile-*fits'))

    # getting all the targetids of the assigned fibers
    print('reading individual fiberassign files')
    favail = list()
    for i_tile, tile_file in enumerate(fba_files):
        if i_tile%50 ==0:
            print(i_tile)
        id_favail, header = fits.getdata(tile_file, 'FIBERASSIGN', header=True)
        favail.extend(id_favail['TARGETID'])
    return np.array(favail)

In [88]:
mtl_names = {0:"strategy_A/targets/gray_subset_dr8_mtl_dark_gray_northern_cap.fits",
             1:"strategy_A/targets/dark0_subset_dr8_mtl_dark_gray_northern_cap.fits",
             2:"strategy_A/targets/dark1_subset_dr8_mtl_dark_gray_northern_cap.fits",
             3:"strategy_A/targets/dark2_dark3_subset_dr8_mtl_dark_gray_northern_cap.fits"}
zcat_names = {0:"strategy_A/zcat/gray_zcat.fits",
             1:"strategy_A/zcat/dark0_zcat.fits",
             2:"strategy_A/zcat/dark1_zcat.fits",
             3:"strategy_A/zcat/dark2_dark3_zcat.fits"}

tile_names = {0:'strategy_A/fiberassign_gray/', 
             1:'strategy_A/fiberassign_dark0/',
             2:'strategy_A/fiberassign_dark1/',
             3:'strategy_A/fiberassign_dark2_dark3/'}

mtl_data = {}
zcat_data = {}
tile_data = {}
for k in mtl_names.keys():
    mtl_data[k] = Table.read(mtl_names[k])
    zcat_data[k] = Table.read(zcat_names[k])
    tile_data[k] = consolidate_fassigned(tile_names[k])

reading individual fiberassign files
0
reading individual fiberassign files
0
reading individual fiberassign files
0
50
reading individual fiberassign files
0
50


In [101]:
def print_stats(id_to_track):
    print("ID_TO_TRACK", id_to_track)
    for k in range(4):
        ii = mtl_data[k]['TARGETID']==id_to_track
        jj = zcat_data[k]['TARGETID']==id_to_track
        kk = np.count_nonzero(tile_data[k]==id_to_track)
        fa = Table([[kk]], names=(['NFIBER']))
        if(np.count_nonzero(jj)):
            a = hstack((mtl_data[k][['PRIORITY', 'NUMOBS_MORE']][ii], fa, zcat_data[k][['NUMOBS']][jj]))
        else:
            a = hstack((mtl_data[k][['PRIORITY', 'NUMOBS_MORE']][ii], fa))      
        print(a)
    print()

In [102]:
ids_to_track = [35185796730325505, 35186182941835863, 35186188885166096, 35186188750949832, 35186194501340726, 35186107519861231]
for id_to_track in ids_to_track:
    print_stats(id_to_track)

ID_TO_TRACK 35185796730325505
PRIORITY NUMOBS_MORE NFIBER NUMOBS
-------- ----------- ------ ------
    3400           4      1      1
PRIORITY NUMOBS_MORE NFIBER NUMOBS
-------- ----------- ------ ------
    3500           3      1      2
PRIORITY NUMOBS_MORE NFIBER NUMOBS
-------- ----------- ------ ------
    3500           2      1      3
PRIORITY NUMOBS_MORE NFIBER NUMOBS
-------- ----------- ------ ------
    3500           1      1      5

ID_TO_TRACK 35186182941835863
PRIORITY NUMOBS_MORE NFIBER
-------- ----------- ------
    3400           4      0
PRIORITY NUMOBS_MORE NFIBER NUMOBS
-------- ----------- ------ ------
    3400           4      1      1
PRIORITY NUMOBS_MORE NFIBER NUMOBS
-------- ----------- ------ ------
    3500           3      1      3
PRIORITY NUMOBS_MORE NFIBER NUMOBS
-------- ----------- ------ ------
    3500           1      1      5

ID_TO_TRACK 35186188885166096


PRIORITY NUMOBS_MORE NFIBER
-------- ----------- ------
    3400           4      0
PRIORITY NUMOBS_MORE NFIBER NUMOBS
-------- ----------- ------ ------
    3400           4      2      2
PRIORITY NUMOBS_MORE NFIBER NUMOBS
-------- ----------- ------ ------
    3500           2      1      3
PRIORITY NUMOBS_MORE NFIBER NUMOBS
-------- ----------- ------ ------
    3500           1      1      5

ID_TO_TRACK 35186188750949832
PRIORITY NUMOBS_MORE NFIBER NUMOBS
-------- ----------- ------ ------
    3400           4      1      1
PRIORITY NUMOBS_MORE NFIBER NUMOBS
-------- ----------- ------ ------
    3500           3      1      2
PRIORITY NUMOBS_MORE NFIBER NUMOBS
-------- ----------- ------ ------
    3500           2      1      3
PRIORITY NUMOBS_MORE NFIBER NUMOBS
-------- ----------- ------ ------
    3500           1      1      5

ID_TO_TRACK 35186194501340726
PRIORITY NUMOBS_MORE NFIBER
-------- ----------- ------
    3400           4      0
PRIORITY NUMOBS_MORE NFIBER NUMOBS
